In [183]:
import pandas as pd
import duckdb

In [185]:
with duckdb.connect('../data/RE_moscow_db.duckdb') as w:
    for table in w.execute('PRAGMA show_tables').df()['name'].values:
        print(table,' :',w.execute(f'select count(*) as events from {table}').df().iloc[0].values[0])

agency_ads  : 3618
builder_ads  : 3830
home_info  : 32933
user_ads  : 4370


In [188]:
with duckdb.connect('../data/general_moscow_home_info.duckdb') as w:
    for table in w.execute('PRAGMA show_tables').df()['name'].values:
        print(table,' :',w.execute(f'select count(*) as events from {table}').df().iloc[0].values[0])

home_base_info  : 32933


In [186]:
with duckdb.connect('../data/RE_moscow_db.duckdb') as w:
    agency_df = w.execute(f'select * from agency_ads').df().assign(ads_type='agency_ads')
    builder_df = w.execute(f'select * from builder_ads').df().assign(ads_type='builder_ads')
    user_df = w.execute(f'select * from user_ads').df().assign(ads_type='user_ads')

In [187]:
agency_df.groupby(pd.qcut(agency_df['m2'],3))['price'].describe().div(1000).round(3).style.format('{:_}')

,count,mean,std,min,25%,50%,75%,max
m2,,,,,,,,
"(29.999, 57.0]",1.286,15_275.868,6_354.98,5_270.673,11_257.5,13_640.0,17_500.0,61_000.0
"(57.0, 77.0]",1.168,24_736.289,18_283.119,6_511.22,15_737.5,20_415.5,28_500.0,192_460.0
"(77.0, 260.0]",1.164,69_580.048,63_579.531,12_130.0,30_000.0,44_945.092,82_000.0,377_328.328
